In [1]:
import pickle

# Replace 'your_file.pkl' with the path to your pickle file
with open('/work/mech-ai-scratch/shreyang/me592/text2nav/embeddings/data/replay_buffer_with_tasks.pkl', 'rb') as f:
    data = pickle.load(f)

In [2]:
# Now 'data' holds your loaded object
print(type(data))

<class 'stable_baselines3.common.buffers.DictReplayBuffer'>


In [3]:
data.observations["rgb"].shape, len(data.observations["tasks"])

((195, 512, 3, 256, 256), 195)

In [31]:
data.observations.keys()

dict_keys(['goal_index', 'poses', 'rgb', 'tasks'])

In [ ]:
for episode_id in range(len(data.observations["tasks"])):
    episode = data.observations["tasks"][episode_id]
    for step_id in range(len(episode)):
            steps = episode[step_id]
            if len(steps)>0:
                for task_id in range(len(steps)):
                    print(episode[step_id][task_id]["embedding"].shape)

In [17]:
# red, green blue, yellow, white

color_index = {
    "red": 0,
    "green": 1,
    "blue": 2,
    "yellow": 3,
    "white": 4
}

colors = []
for episode_id in range(len(data.observations["tasks"])):
    episode = data.observations["tasks"][episode_id]
    for step_id in range(len(episode)):
            steps = episode[step_id]
            if len(steps)>0:
                for task_id in range(len(steps)):
                    colors.append(episode[step_id][task_id]["colour"])
set(colors)

{'blue', 'green', 'red', 'yellow'}

In [ ]:
import numpy as np
embedding_size = 1152

color_index = {
    "red": 0,
    "green": 1,
    "blue": 2,
    "yellow": 3,
    "white": 4
}


all_embeddings = []
for batch_id in range(len(data.observations["tasks"])):
    batch_embeddings = []
    goal_batch = data.observations["goal_index"][batch_id]
    task_batch = data.observations["tasks"][batch_id]

    for image_id, task_list in task_batch.items():
        embedding = None
        if len(task_list) > 0:
            for task in task_list:
                if color_index.get(task.get("colour", None), None) == goal_batch[image_id][0]:
                    embedding = task.get("embedding", None)
        if embedding is None:
            embedding = np.zeros(embedding_size)
        
        batch_embeddings.append(embedding)

    batch_embeddings = np.array(batch_embeddings)
    all_embeddings.append(batch_embeddings)

all_embeddings = np.array(all_embeddings)
print(all_embeddings.shape)

(195, 512, 1152)


In [32]:
# Add the all_embeddings array to the observations
data.observations['embeddings'] = all_embeddings

In [33]:
del data.observations['tasks']

In [34]:
with open('/work/mech-ai-scratch/shreyang/me592/text2nav/embeddings/data/replay_buffer_with_embeddings.pkl', 'wb') as f:
    pickle.dump(data, f)

In [35]:
import pickle

# Replace 'your_file.pkl' with the path to your pickle file
with open('/work/mech-ai-scratch/shreyang/me592/text2nav/embeddings/data/replay_buffer_with_embeddings.pkl', 'rb') as f:
    data = pickle.load(f)

In [36]:
data.observations.keys()

dict_keys(['goal_index', 'poses', 'rgb', 'embeddings'])

In [37]:
data.observations["embeddings"].shape

(195, 512, 1152)

In [44]:
total_nonzero_rows = 0

for embedding in data.observations["embeddings"]:
    arr = embedding
    nonzero_rows = np.sum(np.any(arr != 0, axis=1))
    total_nonzero_rows += nonzero_rows

print(total_nonzero_rows)

24615
